# Transfer of Rights

> **上级页面**: ["Provenance"]("/model/provenance/")

---



In [ ]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Set base_url for clearer output
model.factory.base_url = 'http://test.com/museum/'




## Introduction

Ownership is often more complicated than a single individual acquiring an object and immediately having both physical possession and the legal right of ownership.  This section deals with the description of more complex scenarios that require explicitly describing the right that is held by a particular actor, such as for asymmetric shared ownership of an object. Other scenarios in which a right is being transferred or created rather than the more specific case of the entire right of ownership are also covered here, such as the intellectual property right to perform a work of time-based media or theatre.


## Rights and their Acquisition

The model for acquiring an object (by purchase, gift or other means) incorporates the specific right of ownership into the properties of the `Acquisition` activity.  The acquisition transfers the title of the object between parties, but that "title" could be expanded to be more completely modeled as a `Right`. For some scenarios where we have more information about the interactions and legal status, we need to explicitly model these rights.

Rights are conceptual things that apply to their real world subjects. They are not relative to time, but can be established and invalidated by various activities or events.  The simplest case would be that, as part of the provenance event where an object is acquired, there is also the establishment of the right of ownership of that object by the new owner and the invalidation of the right of ownership of the previous owner.

While it is possible and valid to have both the `Acquisition` and the `Right` in a single provenance entry, it is redundant as the Acquisition model is simpler and more concise without losing any information. The sections below introduce scenarios in which information would be lost without the explicit modeling of the Right.

In order to connect the `Right` to the provenance activity, we introduce a new `Activity` subclass called `RightAcquistion`.  It has two new properties: `establishes` links the `Right` that was established by this activity, and `invalidates` can optionally link to the `Right` that was rendered invalid by the activity.  

Instances of `Right` have two properties: `possessed_by` refers to the actor (`Group` or `Person`) that has the right described, and `applies_to` refers to the thing over which the right pertains.



In [ ]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Provenance entry with explicit rights)
# Who: Museum (new owner), unknown previous owner
# What: ProvenanceEntry representing acquisition with explicit ownership right
# Why: To demonstrate explicit rights modeling alongside acquisition
provenance_entry = vocab.ProvenanceEntry(
    ident="rights_example/1",
    label="Acquisition with Rights"
)

# Step 4: Create related objects and relationships
# What object is involved: A painting
artwork = model.HumanMadeObject(label="Painting")

# Who acquired the object: The museum
owner = model.Group(label="Museum")

# Standard acquisition event
acquisition = model.Acquisition()
acquisition.transferred_title_of = artwork
acquisition.transferred_title_to = owner
provenance_entry.part = acquisition

# Explicit rights acquisition event
rights_acquisition = model.RightAcquisition()
ownership_right = vocab.OwnershipRight(label="Right of Ownership for Painting")
rights_acquisition.establishes = ownership_right
ownership_right.possessed_by = owner
ownership_right.applies_to = artwork
provenance_entry.part = rights_acquisition

# Step 5: Display the generated JSON-LD
print(model.factory.toString(provenance_entry, compact=False))




## Multiple Owners with Different Stakes

A change in ownership might involve more than two parties, resulting in a state where multiple actors together own some share of the object.  Examples of this situation include when art dealers share the costs of purchasing an object and then share the proceeds of sale in the same proportions, when a donor gives part of the value of an object to an institution, or when a marriage is dissolved and the value of the object is split between the individuals.

The base acquisition model allows for ownership to be transferred to multiple actors, but it is impossible to then determine the share of the value that was owned by each party and the implicit assumption should be that all parties owned equal shares. In order to allow for the description of different proportions of the object to be owned by different actors, we need partition the `Right` into shares to be divided. These components are themselves `Right`s that are possessed by the respective parties.

In the same way as describing the physical extent of a physical object, we can use a `Dimension` to describe the logical extent of the ownership. At any given time the total shares of an object should, of course, add up to the entire object.  If this is not true, then it might signify either that the accounting is wrong (more than 100% of the object is claimed as being owned), theft (multiple claims of ownership at the same time, some of which are illegal), or just that it is now unknown what happened.



In [ ]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Provenance entry for shared ownership)
# Who: Dealer 1 (30% owner), Dealer 2 (70% owner)
# What: ProvenanceEntry representing purchase with unequal ownership shares
# Why: To document multiple owners with different proportional stakes
provenance_entry = vocab.ProvenanceEntry(
    ident="shared_ownership/1",
    label="Purchase of Painting by Two Dealers"
)

# Step 4: Create related objects and relationships
# Who: The two dealers
dealer1 = model.Group(label="Dealer 1")
dealer2 = model.Group(label="Dealer 2")

# What object: A painting
artwork = vocab.Painting(label="Painting")

# Measurement unit for percentages
unit = model.MeasurementUnit(
    ident="http://qudt.org/3.0.0/vocab/unit/Percent",
    label="Percent"
)

# Total ownership right (will be divided into shares)
total_right = vocab.OwnershipRight(label="Total Right of Ownership")

# Rights acquisition event
rights_acquisition = model.RightAcquisition()
provenance_entry.part = rights_acquisition
rights_acquisition.establishes = total_right
total_right.applies_to = artwork

# Dealer 1's 30% share
right1 = vocab.OwnershipRight(label="30% Ownership by Dealer 1")
right1.possessed_by = dealer1
dimension1 = model.Dimension()
dimension1.value = 30
dimension1.unit = unit
right1.dimension = dimension1
total_right.part = right1

# Dealer 2's 70% share
right2 = vocab.OwnershipRight(label="70% Ownership by Dealer 2")
right2.possessed_by = dealer2
dimension2 = model.Dimension()
dimension2.value = 70
dimension2.unit = unit
right2.dimension = dimension2
total_right.part = right2

# Step 5: Display the generated JSON-LD
print(model.factory.toString(provenance_entry, compact=False))




### Transfer of Shares

Once the share of an object has been established, that right can be traded as part of future provenance entries.  

In order to maintain consistency with the original establishment of the division of ownership, the difference between the initial state and the new state is not described, but instead the new state is described directly. If a 10% share is acquired from another owner, the 10% is not traded requiring the system to calculate the new total proportions, but instead the new total proportions are described. 

If only existing owners are part of the entry, then there is no need to establish a new Right that is the new total ownership, it is simply a new division of the existing total. Ownership Rights are brought into existence by `Acquisition` events - the set of people who possess the right is fixed, and is the same set of actors as those referenced by `title_transferred_to`. Thus any new Acquisition by a new partial owner requires a new Ownership Right, as described above.




In [ ]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Provenance entry for share trading)
# Who: Dealer 1 (increased to 40%), Dealer 2 (decreased to 60%)
# What: ProvenanceEntry representing trading of ownership shares
# Why: To document how ownership proportions changed between existing owners
provenance_entry = vocab.ProvenanceEntry(
    ident="share_trade/1",
    label="Trading Shares of Painting between Two Dealers"
)

# Step 4: Create related objects and relationships
# Who: The two dealers
dealer1 = model.Group(label="Dealer 1")
dealer2 = model.Group(label="Dealer 2")

# What object: A painting
artwork = vocab.Painting(label="Painting")

# Measurement unit for percentages
unit = model.MeasurementUnit(
    ident="http://qudt.org/3.0.0/vocab/unit/Percent",
    label="Percent"
)

# Dealer 1's new share (40%, up from 30%)
rights_acquisition1 = model.RightAcquisition()
provenance_entry.part = rights_acquisition1
new_right1 = vocab.OwnershipRight(label="40% Ownership by Dealer 1")
new_right1.possessed_by = dealer1
new_right1.applies_to = artwork
dimension1 = model.Dimension()
dimension1.value = 40
dimension1.unit = unit
new_right1.dimension = dimension1
rights_acquisition1.establishes = new_right1
rights_acquisition1.invalidates = vocab.OwnershipRight(
    label="30% Ownership by Dealer 1"
)

# Dealer 2's new share (60%, down from 70%)
rights_acquisition2 = model.RightAcquisition()
provenance_entry.part = rights_acquisition2
new_right2 = vocab.OwnershipRight(label="60% Ownership by Dealer 2")
new_right2.possessed_by = dealer2
dimension2 = model.Dimension()
dimension2.value = 60
dimension2.unit = unit
new_right2.dimension = dimension2
rights_acquisition2.establishes = new_right2
rights_acquisition2.invalidates = vocab.OwnershipRight(
    label="70% Ownership by Dealer 2"
)

# Step 5: Display the generated JSON-LD
print(model.factory.toString(provenance_entry, compact=False))



## Intellectual Property Rights

Other sorts of rights associated with art that can be acquired include the intellectual property rights, such as copyright for a text or image, separate from any physical carrier, or the right to perform some piece of time-based media such as a projected installation.  The manipulation of these rights also uses the `RightAcquisition` pattern.

Many copyrights for images are owned by collective organizations rather than individuals, and these organizations can acquire and exchange those copyrights completely independently of any physical carrier of the image.

__Example:__

A copyright management organization, the Paintings Copyright Association, acquired the copyright for a particular painting in 2014.



In [ ]:
# Step 1: Import cromulent library
from cromulent import model, vocab

# Step 2: Configure factory settings
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# Step 3: Create the main object (Provenance entry for copyright acquisition)
# Who: Paintings Copyright Association (rights organization)
# What: ProvenanceEntry representing acquisition of intellectual property rights
# Why: To document transfer of copyright separate from physical object
provenance_entry = vocab.ProvenanceEntry(
    ident="copyright_acquisition/1",
    label="Copyright acquisition by PCA"
)

# Step 4: Create related objects and relationships
# Who acquired the copyright: PCA organization
pca = model.Group(label="Paintings Copyright Association")

# What artwork the copyright applies to: A painting
artwork = vocab.Painting(label="Painting", art=1)

# Rights acquisition event
rights_acquisition = model.RightAcquisition()
provenance_entry.part = rights_acquisition

# Copyright right being acquired
copyright_right = vocab.CopyrightRight()
rights_acquisition.establishes = copyright_right
copyright_right.possessed_by = pca
copyright_right.applies_to = artwork

# Step 5: Display the generated JSON-LD
print(model.factory.toString(provenance_entry, compact=False))